In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
from keras.utils import to_categorical,normalize
import pandas as pd
import scipy.io as scio
import numpy as np
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from  keras.models import load_model



In [ ]:
data= './icassp/mfcc/Mfccwithoutdct08.mat'
dataset_08= scio.loadmat(data)


In [ ]:
dataset_08['allMfcc'].shape

In [ ]:
data_07 = './icassp/mfcc/Mfccwithoutdct07.mat'
dataset_07= scio.loadmat(data_07)
dataset_07['allMfcc'].shape

In [ ]:
data_09 = './icassp/mfcc/Mfccwithoutdct09.mat'
dataset_09 = scio.loadmat(data_07) 
dataset_09['allMfcc'].shape

In [ ]:
def load_data():
    b=[]
    loc = []
    num = 8
    i_train = 18 
    for j_train in range(1):
        x_box1 = dataset['allMfcc'][i_train][j_train]
            #x_box_true = np.transpose(x_box1[:,:199])
        x_box_true = np.transpose(x_box1)
            #x_box_true = x_box1[:,:190]
        x_box_true = x_box_true.reshape(-1)
        a = x_box_true.tolist()
        for n in range(33):
            m = (n+1)*3*32
            b = a[m-3*32:m]
            b.append(num)
            loc.append(b)                
            #a.append(num)
            #loc.append(a)
    traindata = np.array(loc)
    return traindata

In [ ]:
ture_testdata = load_data()
print(ture_testdata.shape)

In [ ]:
def load_data(traindata):
    np.random.shuffle(traindata)
    testline = int(traindata.shape[0]* (70/100))
    x_train = traindata[:testline , : -1]
    #x_train = normalize(x_train)  #normalize axis 1
    y_train = traindata[:testline, -1]
    x_test = traindata[testline:, : -1]
    #x_test = normalize(x_test)#normalize axis 1
    y_test = traindata[testline:, -1]
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test,10)
    return (x_train, y_train), (x_test, y_test)

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data(ture_testdata)

print(x_train.shape)
print(y_train[0])

In [ ]:
model = load_model('./my_model_32*3_withoutdct_10')

In [ ]:
print(model.summary())

In [ ]:
coldmodel = Sequential()

In [ ]:
for layer in model.layers:
    coldmodel.add(layer)

In [ ]:
coldmodel.summary()

In [ ]:
for layer in coldmodel.layers[:-2]:
    layer.trainable = False

In [ ]:
coldmodel.summary()

In [ ]:
coldmodel.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.00001), metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
coldmodel.fit(x_train, y_train, batch_size=10,epochs=200, validation_split=0.2, callbacks=[early_stopping])
#model.fit(x_train, y_train, batch_size=1,epochs=20)

result = coldmodel.evaluate(x_train, y_train)
print("\nTrain Acc: ", result[1])

result = coldmodel.evaluate(x_test, y_test)
print("\nTest Acc: ", result[1])

classes = coldmodel.predict_classes(x_train[0:1,:])
print(classes)

In [ ]:
coldmodel.save('./my_model_32*3_withoutdct_5_tl_03gy_0928cold')